In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import sys
sys.path.append('/home/aliche/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [12]:
%matplotlib
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as pl
import pickle as pkl
import fnmatch,os
import itertools
import statsmodels.formula.api as sm
from scipy.stats import pearsonr,ttest_ind,ranksums,linregress
import pandas as pd
import seaborn as sns
from signale.tools import locate
import statsmodels.api as sm
from matplotlib.gridspec import GridSpec
from NeuroTools import analysis
from scipy import stats

Using matplotlib backend: Qt5Agg
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
FR_list = sorted([os.path.join(pth[0],pth[1]) for pth in locate('FiringUnits.spk','/home/aliche/dataWork/ali/MECLesion_SleepData/')])

In [96]:
controlIDs = ['3906','3661','614','616','3839']
df = pd.DataFrame([],columns=['Nu','Np','group'])

In [97]:
for item in FR_list:
    NP=0
    NU=0
    pthSplit = item.split('.')[0][:-11]
    print pthSplit
    if pthSplit.endswith('/'):
        fuSPK = pkl.load(open(item,'rb'))
        try:
            plSPK = pkl.load(open(pthSplit+'placeCells.spk','rb'))
            #print plSPK.id_list.size
            NP += plSPK.id_list.size
        except IOError:
            print 'No PL'
        #print fuSPK.id_list.size
        #print '--------------------------------------------'
        NU+=fuSPK.id_list.size
        isControl = np.array([pthSplit.find(tag)>0 for tag in controlIDs]).sum()
        if isControl:
            df = df.append([{'Nu':NU,'Np':NP,'group':'Cnt'}],ignore_index=True)
        else:
            df = df.append([{'Nu':NU,'Np':NP,'group':'Les'}],ignore_index=True)

/home/aliche/dataWork/ali/MECLesion_SleepData/Magda/Rat405/LinearTrack_0-0-0_Day1/parrentData/
/home/aliche/dataWork/ali/MECLesion_SleepData/Magda/Rat434/LinearTrack_0-0-0_Day1/parrentData/
/home/aliche/dataWork/ali/MECLesion_SleepData/Magda/Rat614/LinearTrack_05-31-13_Day1/parrentData/
/home/aliche/dataWork/ali/MECLesion_SleepData/Magda/Rat616/LinearTrack_2013-06-02_Day1/parrentData/
/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-03-2015_Day1/parrentData/
/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3656/LinearTrack_03-04-2015_Day2/parrentData/
/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-03-15Day1/parrentData/
/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-04-15Day2/parrentData/
/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3661/LinearTrack_03-05-15Day3/parrentData/
/home/aliche/dataWork/ali/MECLesion_SleepData/Marta/Rat3754/LinearTrack_07-07-15_Day2/parrentData/
/home/aliche/data

In [90]:
sns.pairplot(df[df.Np>0],hue='group')

In [98]:
df[df.group=='Cnt'].shape,df[df.group=='Les'].shape

((9, 3), (17, 3))

In [29]:
sns.jointplot("Nu","Np",data=df.groupby('group'))#,color=['r','g'])

AttributeError: Cannot access callable attribute 'get' of 'DataFrameGroupBy' objects, try using the 'apply' method

In [17]:
tips = sns.load_dataset("tips")
g = sns.jointplot(x="total_bill", y="tip", data=tips)

In [99]:
df.Np = np.int16(df.Np)
df.Nu = np.int16(df.Nu)

In [103]:
sns.lmplot(x="Nu", y="Np", hue="group", data=df,markers=["o", "x"], palette="Set1",ci=0.0066,fit_reg=True,robust=True)
pl.xlabel('Number of all firing units')
pl.ylabel('Number of place cells')

Text(68.4689,0.5,u'Number of place cells')

In [102]:
from statsmodels.formula.api import ols
llm = ols('Np~Nu',df[df.group=='Les']).fit()
print llm.pvalues,llm.summary()
lx = lambda x:llm.params[1]*x+llm.params[0]
clm = ols('Np~Nu-1',df[df.group=='Cnt']).fit()
print clm.pvalues,clm.summary()
cx = lambda x:clm.params[0]*x

Intercept    3.605311e-02
Nu           5.662635e-08
dtype: float64                             OLS Regression Results                            
Dep. Variable:                     Np   R-squared:                       0.867
Model:                            OLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                     98.11
Date:                 So, 20 Mai 2018   Prob (F-statistic):           5.66e-08
Time:                        21:48:19   Log-Likelihood:                -46.195
No. Observations:                  17   AIC:                             96.39
Df Residuals:                      15   BIC:                             98.06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------

/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/scipy/stats/stats.py:1390: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  "anyway, n=%i" % int(n))
/home/aliche/anaconda3/envs/py27/lib/python2.7/site-packages/scipy/stats/stats.py:1390: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))


In [86]:
xxx = np.array([df[df.group=='Cnt'].Nu.min(),df[df.group=='Cnt'].Nu.max()])
pl.plot(xxx,cx(xxx),c='r')
pl.plot(xxx,lx(xxx))

In [72]:
lx(np.array([0,1,2]))

array([0.        , 0.36488485, 0.7297697 ])

In [55]:

df.max()

Nu       109
Np        52
group    Les
dtype: object

In [76]:
sns.lmplot()

TypeError: lmplot() takes at least 3 arguments (0 given)